### Importazione librerie

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
from selenium.common.exceptions import TimeoutException
import mysql.connector
from requests.exceptions import ReadTimeout
from urllib.error import HTTPError
import collections
import editdistance
from selenium.common.exceptions import WebDriverException as WDE
from instagramy.plugins.analysis import analyze_users_popularity
from selenium.common.exceptions import NoSuchElementException
from sqlalchemy import create_engine

### Connessione al DataBase

In [ ]:
db = mysql.connector.connect(
    host= "localhost",
    user="root",
    password="INSERT PASSWORD",
    database="INSER NAME DB"
)

print(db)
cursor=db.cursor()
#cursor.execute("Create Database ProgettoDataMan")

### Creazione tabelle Spotify e Artista

In [ ]:
#sql = "DROP TABLE Spotify"
#cursor.execute(sql)
cursor.execute("CREATE TABLE Spotify (id INT AUTO_INCREMENT PRIMARY KEY, Canzone VARCHAR(255),Durata_Minuiti VARCHAR(255), Popolarita INT, Artista VARCHAR(255), Popolarita_artista INT, Follower_Artista INT, Url_Artista VARCHAR(255), Album VARCHAR(255))")


In [ ]:
#sql = "DROP TABLE Artista"
#cursor.execute(sql)
cursor.execute("CREATE TABLE Artista (Artista VARCHAR(255) PRIMARY KEY, Popolarita_artista INT, Follower_Artista INT, Url_Artista VARCHAR(255), Numero_di_apparizioni INT, Numero_ascoltatori_mensili VARCHAR(255))")



### Dichiarazione Funzioni

In [ ]:
def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    return tracks

def get_all_url(Url):
    links=[]
    elems = driver.find_elements(By.CSS_SELECTOR, ".Ft1cMGlqDsCbqmXQyeKN [href]")
    for elem in elems:
        link_url= elem.get_attribute('href')
        links.append(link_url)

    l = driver.find_elements(By.CSS_SELECTOR, ".Ft1cMGlqDsCbqmXQyeKN [href]")

    nome_p=[]
    for i in l: 
        nome_playlist= i.get_attribute("title")
        nome_p.append(nome_playlist)
    return links,nome_p

def Playlist(Url,Album,tracks):    
    for track in tracks:
            try:
                if (track["track"])!=None:  
                          track_name = track["track"]["name"]
                          #Duration
                          seconds=(track["track"]["duration_ms"]/1000)%60
                          seconds = int(seconds)
                          minutes=(track["track"]["duration_ms"]/(1000*60))%60
                          minutes = int(minutes)
                          track_duration = ("%d:%d" % (minutes, seconds))

                          #Main Artist
                          artist_uri = track["track"]["artists"][0]["uri"]
                          if(len(track["track"]["artists"][0]["external_urls"]) != 0):
                            artist_url = track["track"]["artists"][0]["external_urls"]["spotify"]
                            artist_info = sp.artist(artist_uri)
                            artist_followers=artist_info["followers"]["total"]



                          #Name, popularity, genre
                            artist_name = track["track"]["artists"][0]["name"]
                            artist_pop = artist_info["popularity"]



                            #Popularity of the track
                            track_pop = track["track"]["popularity"]
                            #values = [track_name,track_duration, track_pop, artist_name, artist_pop,artist_followers,artist_url]
                            #zipped = zip(columns, values)
                            #a_dictionary = dict(zipped)
                            #data.append(a_dictionary)
                            
                            val = (track_name,track_duration, track_pop, artist_name, artist_pop,artist_followers,artist_url,Album)
                            cursor.execute(sql, val)
                            db.commit()
            except ReadTimeout:
                print('Spotify timed out... trying again...')
    return 1


def Is_Certificated_CalcoloEditDistance(Nome_Artista, Nome_Instagram, path):
    try:
        l= WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, path)))
        s= l.text
        print("Element exist")
        nome_distance=(1-(editdistance.eval(Nome_Artista, Nome_Instagram)/max(len(Nome_Artista), len(Nome_Instagram))))+0.5
        
    except NoSuchElementException:
            nome_distance= 1-(editdistance.eval(Nome_Artista, Nome_Instagram)/max(len(Nome_Artista), len(Nome_Instagram)))
            print("Element does not exist")
    except TimeoutException:
            nome_distance= 1-(editdistance.eval(Nome_Artista, Nome_Instagram)/max(len(Nome_Artista), len(Nome_Instagram)))
            print("Element does not exist")
    return nome_distance
    

### Log-in spotify

In [ ]:
PATH = r"/Users/fabriziomerlo/Desktop/Work/chromedriver"
driver = webdriver.Chrome(PATH)
driver.get("https://accounts.spotify.com/it/login")


### Autenticazione per l'utilizzo delle Api

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id='57eb188f9b4240d0a59246fae020b4ec',
client_secret='1e6d7ee6ef2b4f7a832c7b2c02fea9c7')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)
username_spotify = "https://open.spotify.com/user/spotify?si=e020533084944773"

### Scraping in Spotify

In [ ]:
username = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[id=login-username]")))
password = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[id=login-password]")))

username.clear()
username.send_keys("INSERT EMAIL")
password.clear()
password.send_keys("INSERT PASSWORD")

button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[id=login-button]"))).click()

time.sleep(3)

In [ ]:
button2 = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid=web-player-link]"))).click()

In [ ]:
button3= WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[id=onetrust-accept-btn-handler]"))).click()

### Lettura sezioni generi, utilizzati per scaricare il maggior numero di playlist

In [ ]:
df_generi = pd.read_csv("generi.csv")
size = df_generi.url_generi.size

### Caricamento nel DataBase dei dati sulle canzoni contenute nelle Playlist

In [ ]:
nome_playlist=[]
for l in range(size):
    try:
            url_section=df_generi.url_generi[l]
            driver.get(url=url_section)
            time.sleep(15)
            links,nome_p = get_all_url(url_section)
            d = {'url': links, 'playlist': nome_p}
            df2 = pd.DataFrame(data=d)
            size2=df2.url.size
            sql = "INSERT INTO Spotify (Canzone,Durata_Minuiti, Popolarita, Artista,Popolarita_artista, Follower_Artista, Url_Artista, Album) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
            for i in range(size2):
                try:
                    tracks = get_playlist_tracks(username_spotify,str(df2.url[i]))
                    if df2.playlist[i] in nome_playlist:
                        print("Playlist già inserita nel DB")
                    else:
                        nome_playlist.append(df2.playlist[i])
                        print(str(df2.url[i]))
                        print(df2.playlist[i])
                        print(len(tracks))
                        if(len(tracks) > 0 and len(tracks) < 101):
                            x = Playlist(str(df2.url[i]), df2.playlist[i],tracks)
                except spotipy.SpotifyException as ex:
                    print("Spotify find error... trying again...")
                except HTTPError as err:
                    pass
                                                                         

    except ReadTimeout:
        print('Spotify timed out... trying again...')
        for i in range(size2):
            tracks = get_playlist_tracks(username_spotify,str(df2.url[i]))
            print(str(df2.url[i]))
            print(df2.playlist[i])
            x = Playlist(str(df2.url[i]), df2.playlist[i],tracks)        
        



### Lettura nome playlist per arricchimento col nuovo Dataset

In [ ]:
query_Album = "SELECT DISTINCT Album FROM Spotify"

## getting records from the table
cursor.execute(query_Album)

## fetching all records from the 'cursor' object
nome_playlist = cursor.fetchall()

#print (test)
test1 = []
for elem in nome_playlist:
    test1.append(elem[0])
test1



### Arricchimento Dataset


In [ ]:
csv_integration= pd.read_csv("spotify_officialplaylists.csv")
print(csv_integration["Playlist Name"])

In [ ]:
try:
            d = {'url': csv_integration["URL"], 'playlist': csv_integration["Playlist Name"]}
            df2 = pd.DataFrame(data=d)
            size2=df2.url.size
            sql = "INSERT INTO Spotify (Canzone,Durata_Minuiti, Popolarita, Artista,Popolarita_artista, Follower_Artista, Url_Artista, Album) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
            for i in range(size2):
                try:
                    tracks = get_playlist_tracks(username_spotify,str(df2.url[i]))
                    if df2.playlist[i] in test1:
                        print("Playlist già inserita nel DB")
                    else:
                        test1.append(df2.playlist[i])
                        print(str(df2.url[i]))
                        print(df2.playlist[i])
                        print(len(tracks))
                        if(len(tracks) > 0 and len(tracks) < 101):
                            x = Playlist(str(df2.url[i]), df2.playlist[i],tracks)
                except spotipy.SpotifyException as ex:
                    print("Spotify find error... trying again...")
                except HTTPError as err:
                    pass
                                                                         

except ReadTimeout:
        print('Spotify timed out... trying again...')
        for i in range(size2):
            tracks = get_playlist_tracks(username_spotify,str(df2.url[i]))
            print(str(df2.url[i]))
            print(df2.playlist[i])
            x = Playlist(str(df2.url[i]), df2.playlist[i],tracks)   

### Calcolo del numero di volte che appare l'artista nelle playlist scelte da noi di Spotify

In [ ]:
query = "SELECT Artista FROM Spotify"
## getting records from the table
cursor.execute(query)

## fetching all records from the 'cursor' object
numero_apparizioni = cursor.fetchall()

Num_apparizioni_playlist=pd.DataFrame(data=numero_apparizioni)
Num_apparizioni_playlist = Num_apparizioni_playlist.rename(columns={0:'Artista'})
numero_apparizioni=Num_apparizioni_playlist.groupby(['Artista']).size()
Num_apparizioni_playlist=pd.DataFrame(data=numero_apparizioni.values)
Num_apparizioni_playlist['Artista'] = numero_apparizioni.index 
Num_apparizioni_playlist = Num_apparizioni_playlist.rename(columns={0:'Numero_di_apparizioni'})


query = "SELECT DISTINCT Artista FROM Spotify"
## getting records from the table
cursor.execute(query)

## fetching all records from the 'cursor' object
artisti = cursor.fetchall()

Table_Artista= pd.DataFrame(data=artisti)

Table_Artista= Table_Artista.rename(columns={0:'Artista'})
Table_Artista= (pd.merge(Table_Artista, Num_apparizioni_playlist, on='Artista'))
print(Table_Artista)


### Query per selezionare gli artisti e i relativi dati che hanno la popolarità  e i follower maggiori della media

In [ ]:
query = "SELECT DISTINCT Artista, Url_Artista, Popolarita_artista, Follower_Artista FROM Spotify WHERE Popolarita_artista > (SELECT AVG(Popolarita_artista) FROM Spotify) AND Follower_Artista > (SELECT AVG(Follower_Artista) FROM Spotify)"

## getting records from the table
cursor.execute(query)

## fetching all records from the 'cursor' object
records = cursor.fetchall()

Table_Artista_Finale= pd.DataFrame(data=records)
Table_Artista_Finale= Table_Artista_Finale.rename(columns={0: 'Artista', 1:'Url_Artista',2:'Popolarita_artista',3:'Follower_Artista'})
Table_Artista_Finale = (pd.merge(Table_Artista_Finale,Table_Artista,on='Artista'))
print(Table_Artista_Finale)

### Integrazione, tramite scraping, del numero di ascoltatori mensili per ogni cantante 

In [ ]:
Url_Artista = Table_Artista_Finale.Url_Artista.to_list()
Nome_Artista = Table_Artista_Finale.Artista.to_list()
Popolarita_Artista = Table_Artista_Finale.Popolarita_artista.to_list()
Follower_Artista = Table_Artista_Finale.Follower_Artista.to_list()
Numero_di_apparizioni = Table_Artista_Finale.Numero_di_apparizioni.to_list()

size=Table_Artista_Finale.Url_Artista.size
numero_ascolti_mensili=[]
for i in range(size):
    try:
        driver.get(url=Url_Artista[i])
        time.sleep(1)
        sql = "INSERT INTO Artista (Artista, Popolarita_artista, Follower_Artista, Url_Artista, Numero_di_apparizioni, Numero_ascoltatori_Mensili ) VALUES (%s, %s,%s, %s,%s, %s)"
        #num_ascolti = WebDriverWait(driver, 40).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div[2]/div[3]/main/div[2]/div[2]/div/div/div[2]/section/div/div[1]/div[2]/span[3]'))).text
        num_ascolti = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//span[@class='Ydwa1P5GkCggtLlSvphs']"))).text
        print("Artista: " + Nome_Artista[i] + "  " + num_ascolti)
        numero_ascolti_mensili.append(num_ascolti)
        time.sleep(2)
        val=(Nome_Artista[i],Popolarita_Artista[i],Follower_Artista[i],Url_Artista[i],Numero_di_apparizioni[i],num_ascolti)
        cursor.execute(sql, val)
        db.commit()
    except TimeoutException as ex:
        isrunning = 0
        num_ascolti = "NaN"
        print("Artista: " + Nome_Artista[i] + "  " + num_ascolti)
        val=(Nome_Artista[i],Popolarita_Artista[i],Follower_Artista[i],Url_Artista[i],Numero_di_apparizioni[i],num_ascolti)
        cursor.execute(sql, val)
        db.commit()
        numero_ascolti_mensili.append(num_ascolti)
        print("Exception has been thrown")
        
    except WDE:
        time.sleep(5)
            



### Scraping in Instagram per trovare i nomi degli artisti utilizzando il metodo EditDistance per aumentare l'accuratezza

In [ ]:
PATH = r"/Users/fabriziomerlo/Desktop/Work/chromedriver"
driver = webdriver.Chrome(PATH)
driver.get("https://www.instagram.com/")

In [ ]:
accetta_tutti = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Accetta tutti")]'))).click()

In [ ]:
username = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name=username]")))
password = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name=password]")))


#enter username and password
username.clear()
username.send_keys("INSERT EMAIL")
password.clear()
password.send_keys("INSERT PASSWORD")

button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [ ]:
not_now = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Non ora")]'))).click()

In [ ]:
not_now2 = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Non ora")]'))).click()

In [ ]:
session_id = "INSERT SESSION ID"

In [ ]:
query = "SELECT Artista FROM Artista"
## getting records from the table
cursor.execute(query)

## fetching all records from the 'cursor' object
artista = cursor.fetchall()

Artisti=pd.DataFrame(data=artista)
Artisti = Artisti.rename(columns={0:'Artista'})

In [ ]:
Nome_Artista = Artisti.Artista.to_list()
size2=Artisti.Artista.size
nome=[]
for i in range(size):
    searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Cerca']")))
    searchbox.clear()
    #search for the hashtag cat
    keyword = Nome_Artista[i]
    searchbox.send_keys(keyword)
    time.sleep(3)                
    nome1= WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/section/nav/div[2]/div/div/div[2]/div[3]/div/div[2]/div/div[1]/a/div/div[2]/div[1]'))).text
    nome2= WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/section/nav/div[2]/div/div/div[2]/div[3]/div/div[2]/div/div[2]/a/div/div[2]/div[1]'))).text
    nome3= WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/section/nav/div[2]/div/div/div[2]/div[3]/div/div[2]/div/div[3]/a/div/div[2]/div[1]'))).text
    
    nome1_distance = Is_Certificated_CalcoloEditDistance(Nome_Artista[i],nome1,'/html/body/div[1]/section/nav/div[2]/div/div/div[2]/div[3]/div/div[2]/div/div[1]/a/div/div[2]/div[1]/div/div/div[2]/span')+0.4
    nome2_distance = Is_Certificated_CalcoloEditDistance(Nome_Artista[i],nome2,'/html/body/div[1]/section/nav/div[2]/div/div/div[2]/div[3]/div/div[2]/div/div[2]/a/div/div[2]/div[1]/div/div/div[2]/span')
    nome3_distance = Is_Certificated_CalcoloEditDistance(Nome_Artista[i],nome3,'/html/body/div[1]/section/nav/div[2]/div/div/div[2]/div[3]/div/div[2]/div/div[3]/a/div/div[2]/div[1]/div/div/div[2]/span') 
    print(nome1_distance)
    print(nome2_distance)
    print(nome3_distance)
    var = {nome1_distance:"nome1_distance",nome2_distance:"nome2_distance",nome3_distance:"nome3_distance"}
    max_nome= var.get(max(var))
    print(max_nome)
    if (max_nome == "nome1_distance"):
                      nome.append(nome1)
                      val=(nome1)
                      cursor.execute(("UPDATE Artista SET Nome_Instagram = '%s' WHERE id = %s" % (nome1,int(i+1))))
                      print(nome1)
    elif(max_nome == "nome2_distance"):
                      nome.append(nome2)
                      val=(nome2)
                      cursor.execute(("UPDATE Artista SET Nome_Instagram = '%s' WHERE id = %s" % (nome2,int(i+1))))
                      print(nome2)
    else:
                      nome.append(nome3)
                      val=(nome3)
                      cursor.execute(("UPDATE Artista SET Nome_Instagram = '%s' WHERE id = %s" % (nome3,int(i+1))))
                      print(nome3)
    print("---------------------------")
    db.commit()
                      

print(nome)

In [ ]:
query = "select count(*) from Artista where Nome_Instagram like 'Nan';"
## getting records from the table
cursor.execute(query)

## fetching all records from the 'cursor' object
valori_nan = cursor.fetchall()
num_valori_nan = []
for elem in valori_nan:
    num_valori_nan.append(elem[0])
num_valori_nan = num_valori_nan[0]
print(num_valori_nan)

### Selezione della Top 100 dei cantanti, basandoci sul numero di ascoltatori mensili, per trovare tramite API di Instagram il numero di Follower, i Following e i Post

In [ ]:
query ="SELECT Nome_Instagram,Artista,Numero_ascoltatori_mensili FROM ProgettoDataMan.Artista WHERE Nome_Instagram != 'Nan'"
cursor.execute(query)
nome_instagram = cursor.fetchall()

nome=pd.DataFrame(data=nome_instagram)
nome = nome.rename(columns={0:'Nome_Instagram', 1:'Artista', 2:'Numero_ascoltatori_mensili'})
Numero_ascoltatori_mensili = nome.Numero_ascoltatori_mensili.to_list()

result=[]
for i in range(len(Numero_ascoltatori_mensili)):
    result.append(int((Numero_ascoltatori_mensili[i].split(" ", 1)[0]).replace(".","")))

    
nome["Numero_ascoltatori_mensili"]= result
nome = nome.sort_values(by=['Numero_ascoltatori_mensili'],ascending=False)

print(nome)
Nuova_Tabella=nome.iloc[0:100]
nome2= nome.iloc[0:100].Nome_Instagram.to_list()


In [ ]:
session_id = "INSERT SESSION ID" 
dati = analyze_users_popularity(nome2, session_id)
x= pd.DataFrame(data=dati)

In [ ]:
prova = Nuova_Tabella.reset_index(drop=True)
x["Artista"]= prova.Artista

print(x)

### Gestione ridondanza


In [ ]:
cursor.execute("ALTER TABLE Spotify DROP COLUMN Popolarita_artista")
cursor.execute("ALTER TABLE Spotify DROP COLUMN Follower_Artista")
cursor.execute("ALTER TABLE Spotify DROP COLUMN Url_Artista")

### Tabella risultati

In [ ]:
cursor.execute("CREATE TABLE Instagram_Top_100 (Artista VARCHAR(255) PRIMARY KEY, Nome_Instagram VARCHAR(255), Followers INT, Following INT, Posts INT)")



In [ ]:
Nome_Instagram = x.Usernames.to_list()
Followers = x.Followers.to_list()
Following = x.Following.to_list()
Posts = x.Posts.to_list()
Nome_Artista = x.Artista.to_list()
size=x.Artista.size
print(Nome_Instagram)
print(Followers)
print(Following)
print(Posts)
print(Nome_Artista)
for i in range(size):
    sql = "INSERT INTO Instagram_Top_100 (Artista, Nome_Instagram, Followers, Following, Posts) VALUES (%s, %s,%s, %s,%s)"
    val = (Nome_Artista[i],Nome_Instagram[i], Followers[i], Following[i], Posts[i])
    cursor.execute(sql, val)
    db.commit()